# 📖 Catálogo de Dados - Tabelas Amazon Sales

**Projeto:** MVP - Pipeline de Dados de Vendas Amazon  
**Arquitetura:** Medallion (Bronze → Silver → Gold)  
**Modelo:** Star Schema  
**Data:** 14 de Dezembro de 2025  

---

## 📋 Visão Geral das Tabelas

| Camada | Tabela | Tipo | Registros Aprox. | Descrição |
|--------|---------|------|------------------|-----------|
| **Fonte** | `Amazon.csv` | Arquivo CSV | 173 | Dados brutos de vendas da Amazon |
| **Bronze** | `sales_bronze` | Delta Table | 173 | Ingestão raw com metadados |
| **Silver** | `sales_silver` | Delta Table | 173 | Dados limpos e validados |
| **Gold** | `fact_sales` | Fact Table | 173 | Fatos de vendas (Star Schema) |
| **Gold** | `dim_customer` | Dimension | ~150 | Dimensão de clientes |
| **Gold** | `dim_product` | Dimension | ~50 | Dimensão de produtos |
| **Gold** | `dim_seller` | Dimension | ~30 | Dimensão de vendedores |
| **Gold** | `dim_date` | Dimension | ~1400 | Dimensão de tempo |

---

## 📁 **FONTE: Amazon.csv**

**Descrição:** Arquivo CSV contendo dados transacionais de vendas da Amazon  
**Localização:** `/FileStore/shared_uploads/Amazon.csv`  
**Formato:** CSV com header  
**Encoding:** UTF-8  
**Separador:** Vírgula (,)  
**Registros:** 173 pedidos  
**Período:** 2020-2024  

### Estrutura de Campos:

| Campo | Tipo | Descrição | Domínio | Exemplo |
|-------|------|-----------|---------|---------|
| `OrderID` | String | Identificador único do pedido | ORD0000001-ORD0000173 | ORD0000001 |
| `OrderDate` | String | Data do pedido | YYYY-MM-DD | 2023-01-31 |
| `CustomerID` | String | ID do cliente | CUST000001-CUST049999 | CUST001504 |
| `CustomerName` | String | Nome do cliente | Nomes indianos | Vihaan Sharma |
| `ProductID` | String | ID do produto | P00001-P00050 | P00014 |
| `ProductName` | String | Nome do produto | Vários produtos | Drone Mini |
| `Category` | String | Categoria do produto | Books, Electronics, Clothing, etc. | Books |
| `Brand` | String | Marca do produto | BrightLux, UrbanStyle, Zenith, etc. | BrightLux |
| `Quantity` | Integer | Quantidade pedida | 1-5 | 3 |
| `UnitPrice` | Double | Preço unitário | $15.47-$596.80 | 106.59 |
| `Discount` | Double | Taxa de desconto | 0.0-0.3 | 0.0 |
| `Tax` | Double | Valor do imposto | $0.0-$405.63 | 0.0 |
| `ShippingCost` | Double | Custo de frete | $0.09-$14.97 | 0.09 |
| `TotalAmount` | Double | Valor total | $17.70-$3160.43 | 319.86 |
| `PaymentMethod` | String | Método de pagamento | Credit Card, Debit Card, UPI, etc. | Debit Card |
| `OrderStatus` | String | Status do pedido | Delivered, Shipped, Cancelled, etc. | Delivered |
| `City` | String | Cidade do cliente | Cidades americanas | Washington |
| `State` | String | Estado/Província | Estados americanos | DC |
| `Country` | String | País | United States, Canada, India, etc. | India |
| `SellerID` | String | ID do vendedor | SELL00001-SELL01999 | SELL01967 |

**Linhagem:** Arquivo fonte → Bronze Layer  
**Qualidade:** ✅ Sem valores nulos, estrutura consistente

---

## 🥉 **BRONZE: sales_bronze**

**Descrição:** Camada de ingestão raw dos dados Amazon.csv  
**Formato:** Delta Table  
**Localização:** `amazon_sales_mvp.sales_bronze`  
**Estratégia:** Full Load  
**Frequência:** Manual/On-Demand  

### Estrutura de Campos:

| Campo | Tipo | Nullable | Descrição | Transformação |
|-------|------|----------|-----------|---------------|
| `OrderID` | String | ✓ | ID do pedido | Sem alteração |
| `OrderDate` | String | ✓ | Data em formato string | Sem alteração |
| `CustomerID` | String | ✓ | ID do cliente | Sem alteração |
| `CustomerName` | String | ✓ | Nome do cliente | Sem alteração |
| `ProductID` | String | ✓ | ID do produto | Sem alteração |
| `ProductName` | String | ✓ | Nome do produto | Sem alteração |
| `Category` | String | ✓ | Categoria | Sem alteração |
| `Brand` | String | ✓ | Marca | Sem alteração |
| `Quantity` | Integer | ✓ | Quantidade | Sem alteração |
| `UnitPrice` | Double | ✓ | Preço unitário | Sem alteração |
| `Discount` | Double | ✓ | Desconto | Sem alteração |
| `Tax` | Double | ✓ | Taxa | Sem alteração |
| `ShippingCost` | Double | ✓ | Frete | Sem alteração |
| `TotalAmount` | Double | ✓ | Total | Sem alteração |
| `PaymentMethod` | String | ✓ | Pagamento | Sem alteração |
| `OrderStatus` | String | ✓ | Status | Sem alteração |
| `City` | String | ✓ | Cidade | Sem alteração |
| `State` | String | ✓ | Estado | Sem alteração |
| `Country` | String | ✓ | País | Sem alteração |
| `SellerID` | String | ✓ | ID vendedor | Sem alteração |
| **`ingestion_timestamp`** | Timestamp | ✗ | **Timestamp de ingestão** | **Adicionado** |
| **`source_file`** | String | ✗ | **Nome do arquivo fonte** | **Adicionado** |
| **`bronze_layer_version`** | String | ✗ | **Versão da camada** | **Adicionado** |

**Linhagem:** Amazon.csv → sales_bronze  
**SLA:** Sem garantia (desenvolvimento)  
**Retenção:** Permanente

---

## 🥈 **SILVER: sales_silver**

**Descrição:** Camada de dados limpos e validados  
**Formato:** Delta Table  
**Localização:** `amazon_sales_mvp.sales_silver`  
**Estratégia:** Full Load com validações  
**Frequência:** Após atualização Bronze  

### Estrutura de Campos:

| Campo | Tipo | Nullable | Descrição | Validação/Transformação |
|-------|------|----------|-----------|-------------------------|
| `order_id` | String | ✗ | ID do pedido | Snake_case |
| `customer_id` | String | ✗ | ID do cliente | Snake_case |
| `product_id` | String | ✗ | ID do produto | Snake_case |
| `seller_id` | String | ✗ | ID do vendedor | Snake_case |
| **`order_date`** | Date | ✗ | **Data convertida** | **String → Date** |
| **`order_year`** | Integer | ✗ | **Ano extraído** | **year(order_date)** |
| **`order_month`** | Integer | ✗ | **Mês extraído** | **month(order_date)** |
| **`order_day_of_week`** | Integer | ✗ | **Dia da semana** | **dayofweek(order_date)** |
| `customer_name` | String | ✓ | Nome do cliente | UPPER + TRIM |
| `city` | String | ✓ | Cidade | UPPER + TRIM |
| `state` | String | ✓ | Estado | UPPER + TRIM |
| `country` | String | ✓ | País | UPPER + TRIM |
| `product_name` | String | ✓ | Nome do produto | TRIM |
| `category` | String | ✓ | Categoria | UPPER + TRIM |
| `brand` | String | ✓ | Marca | UPPER + TRIM |
| `quantity` | Integer | ✗ | Quantidade | Validado > 0, default 1 |
| `unit_price` | Double | ✗ | Preço unitário | Validado ≥ 0 |
| `discount_rate` | Double | ✗ | Taxa de desconto | Validado 0-1 |
| `tax_amount` | Double | ✗ | Valor do imposto | Validado ≥ 0 |
| `shipping_cost` | Double | ✗ | Custo de frete | Validado ≥ 0 |
| `total_amount` | Double | ✗ | Valor total | Validado > 0 |
| `payment_method` | String | ✓ | Método pagamento | UPPER + TRIM |
| `order_status` | String | ✓ | Status do pedido | UPPER + TRIM |
| **`gross_amount`** | Double | ✗ | **Valor bruto calculado** | **unit_price × quantity** |
| **`discount_amount`** | Double | ✗ | **Valor desconto calculado** | **gross_amount × discount_rate** |
| **`net_amount`** | Double | ✗ | **Valor líquido calculado** | **total_amount - tax - shipping** |
| **`is_suspicious_amount`** | Integer | ✗ | **Flag valor suspeito** | **1 se total_amount ≤ 0** |
| **`is_failed_order`** | Integer | ✗ | **Flag pedido falhou** | **1 se status = CANCELLED/RETURNED** |
| **`silver_processed_timestamp`** | Timestamp | ✗ | **Timestamp processamento** | **current_timestamp()** |
| **`silver_layer_version`** | String | ✗ | **Versão da camada** | **"1.0"** |

**Linhagem:** sales_bronze → sales_silver  
**Regras de Qualidade:**
- ✅ Datas válidas (não nulas)
- ✅ Valores financeiros não negativos
- ✅ Quantidades positivas
- ✅ Padronização de campos texto

**SLA:** Processamento em até 5 minutos após Bronze

---

## 🥇 **GOLD: fact_sales** (Tabela Fato)

**Descrição:** Tabela fato central do modelo dimensional Star Schema  
**Formato:** Delta Table  
**Localização:** `amazon_sales_mvp.fact_sales`  
**Granularidade:** 1 linha por item de pedido  
**Tipo:** Transaction Fact Table  

### Estrutura de Campos:

| Campo | Tipo | Nullable | Descrição | Propósito |
|-------|------|----------|-----------|-----------|
| **`order_key`** | String | ✗ | **Chave do pedido** | **Chave degenerada** |
| **`date_key`** | Integer | ✗ | **FK para dim_date** | **YYYYMMDD** |
| **`customer_key`** | String | ✗ | **FK para dim_customer** | **customer_id** |
| **`product_key`** | String | ✗ | **FK para dim_product** | **product_id** |
| **`seller_key`** | String | ✗ | **FK para dim_seller** | **seller_id** |
| **Métricas Numéricas:** |  |  |  |  |
| `quantity` | Integer | ✗ | Quantidade vendida | Métrica aditiva |
| `unit_price` | Double | ✗ | Preço unitário | Métrica semi-aditiva |
| `gross_amount` | Double | ✗ | Valor bruto | Métrica aditiva |
| `discount_rate` | Double | ✗ | Taxa de desconto | Métrica não-aditiva |
| `discount_amount` | Double | ✗ | Valor do desconto | Métrica aditiva |
| `tax_amount` | Double | ✗ | Valor do imposto | Métrica aditiva |
| `shipping_cost` | Double | ✗ | Custo de frete | Métrica aditiva |
| `net_amount` | Double | ✗ | Valor líquido | Métrica aditiva |
| `total_amount` | Double | ✗ | **Valor total (principal)** | **Métrica aditiva** |
| **Dimensões Degeneradas:** |  |  |  |  |
| `payment_method` | String | ✓ | Método de pagamento | Atributo transacional |
| `order_status` | String | ✓ | Status do pedido | Atributo transacional |
| **Flags de Qualidade:** |  |  |  |  |
| `is_suspicious_amount` | Integer | ✗ | Flag de valor suspeito | Indicador qualidade |
| `is_failed_order` | Integer | ✗ | Flag de pedido falhado | Indicador qualidade |
| **Metadados:** |  |  |  |  |
| `fact_created_timestamp` | Timestamp | ✗ | Timestamp criação | Auditoria |

**Linhagem:** sales_silver → fact_sales  
**Relacionamentos:**
- → dim_date (date_key)
- → dim_customer (customer_key)  
- → dim_product (product_key)
- → dim_seller (seller_key)

**KPIs Suportados:**
- 📊 Receita Total: `SUM(total_amount)`
- 📊 Ticket Médio: `AVG(total_amount)`  
- 📊 Volume Vendas: `SUM(quantity)`
- 📊 Margem Desconto: `SUM(discount_amount)`

---

## 🥇 **GOLD: dim_customer** (Dimensão Cliente)

**Descrição:** Dimensão de clientes com informações agregadas e segmentação  
**Formato:** Delta Table  
**Localização:** `amazon_sales_mvp.dim_customer`  
**Tipo:** Type 1 SCD (Slowly Changing Dimension)  

### Estrutura de Campos:

| Campo | Tipo | Nullable | Descrição | Cálculo/Fonte |
|-------|------|----------|-----------|---------------|
| **`customer_id`** | String | ✗ | **Chave primária** | **Identificador único** |
| `customer_name` | String | ✓ | Nome do cliente | Silver: customer_name |
| `city` | String | ✓ | Cidade | Silver: city |
| `state` | String | ✓ | Estado/Província | Silver: state |
| `country` | String | ✓ | País | Silver: country |
| **Métricas Agregadas:** |  |  |  |  |
| `first_order_date` | Date | ✗ | Data primeiro pedido | MIN(order_date) |
| `last_order_date` | Date | ✗ | Data último pedido | MAX(order_date) |
| `total_orders` | Long | ✗ | Total de pedidos | COUNT(order_id) |
| `total_lifetime_value` | Double | ✗ | **Valor total gasto** | **SUM(total_amount)** |
| **Segmentação de Negócio:** |  |  |  |  |
| **`customer_segment`** | String | ✗ | **Segmento do cliente** | **VIP/HIGH/MEDIUM/LOW_VALUE** |

**Regras de Segmentação:**
- 🌟 **VIP**: Lifetime Value > $2,000
- 🔥 **HIGH_VALUE**: Lifetime Value > $1,000
- 📈 **MEDIUM_VALUE**: Lifetime Value > $500
- 📊 **LOW_VALUE**: Lifetime Value ≤ $500

**Linhagem:** sales_silver → dim_customer  
**Atualização:** Recalculado a cada refresh Silver

---

## 🥇 **GOLD: dim_product** (Dimensão Produto)

**Descrição:** Dimensão de produtos com métricas de performance e categorização  
**Formato:** Delta Table  
**Localização:** `amazon_sales_mvp.dim_product`  
**Tipo:** Type 1 SCD  

### Estrutura de Campos:

| Campo | Tipo | Nullable | Descrição | Cálculo/Fonte |
|-------|------|----------|-----------|---------------|
| **`product_id`** | String | ✗ | **Chave primária** | **Identificador único** |
| `product_name` | String | ✓ | Nome do produto | Silver: product_name |
| `category` | String | ✓ | Categoria | Silver: category |
| `brand` | String | ✓ | Marca | Silver: brand |
| **Métricas de Performance:** |  |  |  |  |
| `avg_unit_price` | Double | ✗ | Preço médio unitário | AVG(unit_price) |
| `total_sales` | Long | ✗ | Total de vendas | COUNT(order_id) |
| `total_quantity_sold` | Long | ✗ | Quantidade total vendida | SUM(quantity) |
| **Categorização de Preço:** |  |  |  |  |
| **`price_category`** | String | ✗ | **Categoria de preço** | **PREMIUM/MID_RANGE/BUDGET** |

**Regras de Categorização de Preço:**
- 💎 **PREMIUM**: Preço médio > $400
- 🏆 **MID_RANGE**: Preço médio $200-$400
- 💰 **BUDGET**: Preço médio < $200

**Linhagem:** sales_silver → dim_product

---

## 🥇 **GOLD: dim_seller** (Dimensão Vendedor)

**Descrição:** Dimensão de vendedores com métricas de performance  
**Formato:** Delta Table  
**Localização:** `amazon_sales_mvp.dim_seller`  
**Tipo:** Type 1 SCD  

### Estrutura de Campos:

| Campo | Tipo | Nullable | Descrição | Cálculo/Fonte |
|-------|------|----------|-----------|---------------|
| **`seller_id`** | String | ✗ | **Chave primária** | **Identificador único** |
| **Métricas de Performance:** |  |  |  |  |
| `unique_customers` | Long | ✗ | Clientes únicos | COUNT(DISTINCT customer_id) |
| `unique_products` | Long | ✗ | Produtos únicos | COUNT(DISTINCT product_id) |
| `total_orders` | Long | ✗ | Total de pedidos | COUNT(order_id) |
| `total_revenue` | Double | ✗ | **Receita total** | **SUM(total_amount)** |
| `avg_order_value` | Double | ✗ | Ticket médio | AVG(total_amount) |
| **Classificação de Performance:** |  |  |  |  |
| **`seller_performance`** | String | ✗ | **Nível de performance** | **TOP/HIGH/STANDARD** |

**Regras de Performance:**
- 🏆 **TOP_PERFORMER**: Receita > $10,000
- 🔥 **HIGH_PERFORMER**: Receita > $5,000
- 📊 **STANDARD**: Receita ≤ $5,000

**Linhagem:** sales_silver → dim_seller

---

## 🥇 **GOLD: dim_date** (Dimensão Temporal)

**Descrição:** Dimensão de tempo com hierarquias temporais completas  
**Formato:** Delta Table  
**Localização:** `amazon_sales_mvp.dim_date`  
**Tipo:** Dimensão de Tempo Padrão  
**Cobertura:** Todas as datas presentes nos dados de pedidos  

### Estrutura de Campos:

| Campo | Tipo | Nullable | Descrição | Exemplo | Uso |
|-------|------|----------|-----------|---------|-----|
| **`date_key`** | Integer | ✗ | **Chave primária** | **20231231** | **Join com fact** |
| `order_date` | Date | ✗ | Data completa | 2023-12-31 | Filtros de data |
| **Hierarquia Temporal:** |  |  |  |  |  |
| `year` | Integer | ✗ | Ano | 2023 | Agrupamento anual |
| `quarter` | Integer | ✗ | Trimestre | 4 | Análise trimestral |
| `month` | Integer | ✗ | Mês (número) | 12 | Agrupamento mensal |
| `month_name` | String | ✗ | Nome do mês | December | Visualizações |
| `day` | Integer | ✗ | Dia do mês | 31 | Análise diária |
| `day_of_week` | Integer | ✗ | Dia da semana (1-7) | 1 | Análise semanal |
| `day_name` | String | ✗ | Nome do dia | Sunday | Visualizações |
| **Flags de Negócio:** |  |  |  |  |  |
| **`is_weekend`** | Integer | ✗ | **Flag fim de semana** | **1** | **Análise comportamental** |

**Características Especiais:**
- 📅 **Formato de Chave:** YYYYMMDD (Integer para performance)
- 🔄 **Granularidade:** Diária  
- 📊 **Hierarquias:** Ano → Trimestre → Mês → Dia
- 🎯 **Otimizada para:** Agregações temporais e filtros de período

**Linhagem:** sales_silver.order_date → dim_date  
**Casos de Uso:**
- Análises de sazonalidade
- Comparações período sobre período  
- Filtros de dashboard por tempo
- Agrupamentos temporais em relatórios

---

## 🔗 **Relacionamentos do Modelo Dimensional**

```
                    dim_date
                   (date_key)
                       ↑
                       │
dim_customer ← fact_sales → dim_product
(customer_key)    │         (product_key)
                  ↓
               dim_seller
              (seller_key)
```

**Cardinalidades:**
- **fact_sales : dim_date** = N:1 (Muitos pedidos por data)
- **fact_sales : dim_customer** = N:1 (Muitos pedidos por cliente)  
- **fact_sales : dim_product** = N:1 (Muitos pedidos por produto)
- **fact_sales : dim_seller** = N:1 (Muitos pedidos por vendedor)

**Integridade Referencial:**
- ✅ Todas as FKs em fact_sales possuem registros correspondentes nas dimensões
- ✅ Não há registros órfãos
- ✅ Chaves primárias são únicas em todas as dimensões

---

## 📊 **Métricas de Qualidade do Modelo**

### ✅ **Completude**
- **Bronze Layer:** 100% dos registros fonte preservados
- **Silver Layer:** 100% dos registros validados e limpos
- **Gold Layer:** 100% dos relacionamentos íntegros

### ✅ **Consistência**
- **Padronização:** Todos os campos texto em formato consistente
- **Tipos de Dados:** Conversões validadas e sem erros
- **Domínios:** Valores dentro dos domínios esperados

### ✅ **Integridade**
- **Referencial:** Todas as FKs válidas
- **Entidade:** PKs únicos em todas as tabelas
- **Negócio:** Regras de negócio aplicadas corretamente

### 📈 **Performance**
- **Particionamento:** Tabelas otimizadas por data
- **Indexação:** Chaves primárias e estrangeiras indexadas
- **Compressão:** Delta Lake com compressão automática

---

## 🎯 **Casos de Uso Suportados**

### 📊 **Análises de Vendas**
- Volume de vendas por período
- Performance de produtos e categorias
- Impacto de descontos nas vendas

### 👥 **Análise de Clientes**
- Segmentação de clientes por valor
- Comportamento de compra por região
- Análise de lifetime value

### 🛍️ **Análise de Produtos**
- Produtos mais vendidos
- Performance por categoria e marca
- Análise de preços

### 🕐 **Análise Temporal**
- Sazonalidade de vendas
- Comparações período sobre período
- Tendências de crescimento

### 💳 **Análise Operacional**
- Métodos de pagamento mais utilizados
- Taxa de cancelamento e devolução
- Performance de vendedores

In [ ]:
# Código Python para Geração Automática de Relatório do Catálogo
from datetime import datetime

def generate_catalog_summary():
    """
    Gera relatório executivo do catálogo de dados
    """
    print("📖 CATÁLOGO DE DADOS - RELATÓRIO EXECUTIVO")
    print("=" * 60)
    
    # Configurações do projeto
    DATABASE_NAME = "amazon_sales_mvp"
    
    # Informações das tabelas
    tables_info = [
        ("Fonte", "Amazon.csv", "CSV File", 173, "Dados brutos de vendas"),
        ("Bronze", "sales_bronze", "Delta Table", 173, "Ingestão raw com metadados"),
        ("Silver", "sales_silver", "Delta Table", 173, "Dados limpos e validados"),
        ("Gold", "fact_sales", "Fact Table", 173, "Fatos de vendas"),
        ("Gold", "dim_customer", "Dimension", "~150", "Dimensão de clientes"),
        ("Gold", "dim_product", "Dimension", "~50", "Dimensão de produtos"),
        ("Gold", "dim_seller", "Dimension", "~30", "Dimensão de vendedores"),
        ("Gold", "dim_date", "Dimension", "~1400", "Dimensão de tempo")
    ]
    
    print(f"\n🏢 DATABASE: {DATABASE_NAME}")
    print(f"📍 Localização: Databricks Workspace")
    print(f"🏗️ Arquitetura: Medallion (Bronze → Silver → Gold)")
    print(f"📊 Modelo: Star Schema")
    
    print(f"\n📋 INVENTÁRIO DE TABELAS:")
    print(f"{'Layer':<8} | {'Table':<20} | {'Type':<12} | {'Records':<10} | {'Description':<25}")
    print("-" * 85)
    
    for layer, table_name, table_type, records, description in tables_info:
        print(f"{layer:<8} | {table_name:<20} | {table_type:<12} | {records:<10} | {description:<25}")
    
    print(f"\n🔗 LINHAGEM DE DADOS:")
    print(f"┌─ Amazon.csv (Fonte)")
    print(f"│   └─ sales_bronze (Ingestão)")
    print(f"│       └─ sales_silver (Limpeza & Validação)")
    print(f"│           ├─ fact_sales (Tabela Fato)")
    print(f"│           ├─ dim_customer (Dim Cliente)")
    print(f"│           ├─ dim_product (Dim Produto)")
    print(f"│           ├─ dim_seller (Dim Vendedor)")
    print(f"│           └─ dim_date (Dim Tempo)")
    
    print(f"\n✅ CARACTERÍSTICAS DO MODELO:")
    print(f"• Arquitetura: Medallion com 3 camadas")
    print(f"• Modelo Dimensional: Star Schema")
    print(f"• Granularidade: 1 linha por item de pedido")
    print(f"• Integridade: Todas as FKs válidas")
    print(f"• Qualidade: Validações em todas as camadas")
    
    print(f"\n🎯 CASOS DE USO SUPORTADOS:")
    print(f"• Análises de vendas e performance")
    print(f"• Segmentação de clientes")
    print(f"• Análise temporal e sazonalidade")
    print(f"• Métricas operacionais")
    print(f"• Dashboards e relatórios executivos")
    
    print(f"\n📅 Catálogo gerado em: {datetime.now()}")
    print("✅ Documentação completa disponível acima!")

# Executar relatório
generate_catalog_summary()

---

## 🏗️ Diagrama Entidade-Relacionamento (DER)

### 📊 Modelo Dimensional Star Schema

O modelo dimensional implementado segue a arquitetura Star Schema, com uma tabela fato central (`fact_sales`) conectada às dimensões através de chaves estrangeiras.

#### 🎯 Estrutura do DER:

```
                    📅 dim_date
                   ┌─────────────────┐
                   │ date_key (PK)   │◄─┐
                   │ order_date      │  │
                   │ year            │  │
                   │ quarter         │  │
                   │ month           │  │ FK: date_key
                   │ month_name      │  │
                   │ day             │  │
                   │ day_of_week     │  │
                   │ day_name        │  │
                   │ is_weekend      │  │
                   └─────────────────┘  │
                                        │
👤 dim_customer                        │               📦 dim_product
┌─────────────────┐                     │              ┌─────────────────┐
│ customer_id(PK) │◄─┐                  │          ┌──►│ product_id (PK) │
│ customer_name   │  │                  │          │   │ product_name    │
│ city            │  │                  │          │   │ category        │
│ state           │  │                  │          │   │ brand           │
│ country         │  │FK: customer_key  │          │   │ avg_unit_price  │
│ first_order_date│  │                  │          │   │ total_sales     │
│ last_order_date │  │                  │          │   │ total_qty_sold  │
│ total_orders    │  │                  │          │   │ price_category  │
│ lifetime_value  │  │                  │          │   └─────────────────┘
│ customer_segment│  │                  │          │FK: product_key
└─────────────────┘  │                  │          │
                     │                  │          │
                     │     💰 fact_sales │          │
                     │    ┌─────────────────┐      │
                     └────┤ order_key       │      │
                          │ date_key    (FK)├──────┘
                          │ customer_key(FK)├──────┘
                          │ product_key (FK)├──────┘
                          │ seller_key  (FK)├──────┐
                          │                 │      │
                          │ ── MÉTRICAS ──  │      │
                          │ quantity        │      │
                          │ unit_price      │      │
                          │ gross_amount    │      │
                          │ discount_rate   │      │
                          │ discount_amount │      │
                          │ tax_amount      │      │
                          │ shipping_cost   │      │
                          │ net_amount      │      │
                          │ total_amount    │      │
                          │                 │      │
                          │ ── DIMENSÕES ── │      │
                          │ payment_method  │      │
                          │ order_status    │      │
                          │                 │      │
                          │ ── QUALIDADE ── │      │
                          │ is_suspicious   │      │
                          │ is_failed_order │      │
                          └─────────────────┘      │
                                                   │
                                                   │FK: seller_key
                          🏪 dim_seller              │
                         ┌─────────────────┐       │
                         │ seller_id (PK)  │◄──────┘
                         │ unique_customers│
                         │ unique_products │
                         │ total_orders    │
                         │ total_revenue   │
                         │ avg_order_value │
                         │ seller_perform. │
                         └─────────────────┘
```

#### 🔗 Relacionamentos e Cardinalidades:

| Relacionamento | Cardinalidade | Descrição |
|----------------|---------------|-----------|
| **fact_sales ➤ dim_date** | N:1 | Muitos pedidos por data |
| **fact_sales ➤ dim_customer** | N:1 | Muitos pedidos por cliente |
| **fact_sales ➤ dim_product** | N:1 | Muitos pedidos por produto |
| **fact_sales ➤ dim_seller** | N:1 | Muitos pedidos por vendedor |

#### 🛡️ Integridade Referencial:

- ✅ **Chaves Primárias:** Únicas em todas as dimensões
- ✅ **Chaves Estrangeiras:** Todas têm correspondência nas dimensões
- ✅ **Sem Órfãos:** Nenhum registro sem referência válida
- ✅ **Constraints:** Implementadas via validações no Silver Layer

In [ ]:
# Geração Automática do DER usando Python
import matplotlib.pyplot as plt
import matplotlib.patches as patches
from matplotlib.patches import FancyBboxPatch, ConnectionPatch
import numpy as np

def create_erd_diagram():
    """
    Cria um Diagrama Entidade-Relacionamento visual usando matplotlib
    """
    
    fig, ax = plt.subplots(1, 1, figsize=(16, 12))
    ax.set_xlim(0, 100)
    ax.set_ylim(0, 100)
    ax.axis('off')
    
    # Cores para diferentes tipos de tabelas
    colors = {
        'fact': '#FF6B6B',      # Vermelho para tabela fato
        'dimension': '#4ECDC4',  # Verde-água para dimensões
        'header': '#45B7D1'      # Azul para cabeçalhos
    }
    
    # Função para criar uma tabela
    def create_table(ax, x, y, width, height, title, fields, table_type='dimension'):
        # Caixa principal
        box = FancyBboxPatch(
            (x, y), width, height,
            boxstyle="round,pad=0.02",
            facecolor=colors[table_type],
            edgecolor='black',
            linewidth=2,
            alpha=0.8
        )
        ax.add_patch(box)
        
        # Cabeçalho da tabela
        header_box = FancyBboxPatch(
            (x, y + height - 8), width, 8,
            boxstyle="round,pad=0.02",
            facecolor=colors['header'],
            edgecolor='black',
            linewidth=1,
            alpha=0.9
        )
        ax.add_patch(header_box)
        
        # Título da tabela
        ax.text(x + width/2, y + height - 4, title, 
                ha='center', va='center', fontweight='bold', 
                fontsize=10, color='white')
        
        # Campos da tabela
        field_height = (height - 8) / len(fields)
        for i, field in enumerate(fields):
            field_y = y + height - 8 - (i + 1) * field_height + field_height/2
            
            # Destacar chaves primárias e estrangeiras
            if '(PK)' in field:
                ax.text(x + 2, field_y, field, ha='left', va='center', 
                       fontweight='bold', fontsize=8, color='#2C3E50')
            elif '(FK)' in field:
                ax.text(x + 2, field_y, field, ha='left', va='center', 
                       fontweight='bold', fontsize=8, color='#E74C3C')
            else:
                ax.text(x + 2, field_y, field, ha='left', va='center', 
                       fontsize=8, color='#2C3E50')
        
        return (x + width/2, y + height/2)  # Retorna o centro da tabela
    
    # Definir posições e criar tabelas
    
    # 1. TABELA FATO (centro)
    fact_fields = [
        'order_key',
        'date_key (FK)',
        'customer_key (FK)', 
        'product_key (FK)',
        'seller_key (FK)',
        '─────────────',
        'quantity',
        'unit_price',
        'gross_amount',
        'discount_rate',
        'discount_amount', 
        'tax_amount',
        'shipping_cost',
        'total_amount',
        '─────────────',
        'payment_method',
        'order_status',
        'is_suspicious',
        'is_failed_order'
    ]
    
    fact_center = create_table(ax, 35, 35, 30, 30, '💰 FACT_SALES', fact_fields, 'fact')
    
    # 2. DIM_DATE (superior)
    date_fields = [
        'date_key (PK)',
        'order_date',
        'year',
        'quarter', 
        'month',
        'month_name',
        'day',
        'day_of_week',
        'day_name',
        'is_weekend'
    ]
    
    date_center = create_table(ax, 35, 75, 30, 20, '📅 DIM_DATE', date_fields)
    
    # 3. DIM_CUSTOMER (esquerda)
    customer_fields = [
        'customer_id (PK)',
        'customer_name',
        'city',
        'state',
        'country',
        'first_order_date',
        'last_order_date', 
        'total_orders',
        'lifetime_value',
        'customer_segment'
    ]
    
    customer_center = create_table(ax, 5, 45, 25, 20, '👤 DIM_CUSTOMER', customer_fields)
    
    # 4. DIM_PRODUCT (direita)
    product_fields = [
        'product_id (PK)',
        'product_name',
        'category',
        'brand',
        'avg_unit_price',
        'total_sales',
        'total_qty_sold',
        'price_category'
    ]
    
    product_center = create_table(ax, 70, 45, 25, 16, '📦 DIM_PRODUCT', product_fields)
    
    # 5. DIM_SELLER (inferior)
    seller_fields = [
        'seller_id (PK)',
        'unique_customers',
        'unique_products',
        'total_orders',
        'total_revenue',
        'avg_order_value',
        'seller_performance'
    ]
    
    seller_center = create_table(ax, 35, 5, 30, 14, '🏪 DIM_SELLER', seller_fields)
    
    # Função para criar conexões
    def create_connection(start, end, label='', offset=0):
        # Calcular pontos de conexão
        dx = end[0] - start[0]
        dy = end[1] - start[1]
        
        # Ajustar pontos para as bordas das caixas
        if abs(dx) > abs(dy):  # Conexão horizontal
            if dx > 0:  # Para a direita
                start_point = (start[0] + 15, start[1] + offset)
                end_point = (end[0] - 12.5, end[1] + offset)
            else:  # Para a esquerda
                start_point = (start[0] - 15, start[1] + offset)
                end_point = (end[0] + 12.5, end[1] + offset)
        else:  # Conexão vertical
            if dy > 0:  # Para cima
                start_point = (start[0] + offset, start[1] + 15)
                end_point = (end[0] + offset, end[1] - 10)
            else:  # Para baixo
                start_point = (start[0] + offset, start[1] - 15)
                end_point = (end[0] + offset, end[1] + 7)
        
        # Criar a linha de conexão
        con = ConnectionPatch(
            start_point, end_point, "data", "data",
            arrowstyle="->", shrinkA=0, shrinkB=0,
            mutation_scale=15, fc="red", ec="red", linewidth=2
        )
        ax.add_patch(con)
        
        # Adicionar label se fornecido
        if label:
            mid_x = (start_point[0] + end_point[0]) / 2
            mid_y = (start_point[1] + end_point[1]) / 2
            ax.text(mid_x, mid_y + 2, label, ha='center', va='bottom', 
                   fontsize=8, color='red', fontweight='bold',
                   bbox=dict(boxstyle="round,pad=0.3", facecolor='white', alpha=0.8))
    
    # Criar as conexões (relacionamentos)
    create_connection(fact_center, date_center, '1:N', -3)
    create_connection(fact_center, customer_center, '1:N', 0)
    create_connection(fact_center, product_center, '1:N', 0)
    create_connection(fact_center, seller_center, '1:N', 3)
    
    # Título do diagrama
    ax.text(50, 95, 'DIAGRAMA ENTIDADE-RELACIONAMENTO (DER)\nModelo Star Schema - Amazon Sales MVP', 
            ha='center', va='center', fontsize=16, fontweight='bold',
            bbox=dict(boxstyle="round,pad=0.5", facecolor='lightblue', alpha=0.8))
    
    # Legenda
    legend_elements = [
        patches.Patch(color=colors['fact'], label='Tabela Fato'),
        patches.Patch(color=colors['dimension'], label='Tabelas Dimensão'),
        patches.Patch(color='red', label='Relacionamento 1:N')
    ]
    
    ax.legend(handles=legend_elements, loc='upper right', bbox_to_anchor=(0.98, 0.15))
    
    # Adicionar informações técnicas
    tech_info = """
    INFORMAÇÕES TÉCNICAS:
    • Modelo: Star Schema
    • Granularidade: 1 linha por item de pedido
    • Relacionamentos: Todos 1:N (One-to-Many)
    • Integridade: Chaves estrangeiras validadas
    • Performance: Otimizado para consultas analíticas
    """
    
    ax.text(2, 25, tech_info, ha='left', va='top', fontsize=9,
            bbox=dict(boxstyle="round,pad=0.5", facecolor='lightyellow', alpha=0.8))
    
    plt.tight_layout()
    plt.title('Amazon Sales Data Warehouse - Entity Relationship Diagram', 
              fontsize=18, fontweight='bold', pad=20)
    
    return fig

# Gerar e exibir o DER
print("🏗️ GERANDO DIAGRAMA ENTIDADE-RELACIONAMENTO...")
print("=" * 50)

try:
    fig = create_erd_diagram()
    plt.show()
    print("✅ DER gerado com sucesso!")
    print("\n📊 CARACTERÍSTICAS DO MODELO:")
    print("• Arquitetura: Star Schema")
    print("• Tabela Fato: fact_sales (centro)")
    print("• Dimensões: 4 tabelas (date, customer, product, seller)")
    print("• Relacionamentos: Todos 1:N otimizados para analytics")
    print("• Integridade: Chaves validadas em todas as camadas")
    
except Exception as e:
    print(f"❌ Erro ao gerar DER: {str(e)}")
    print("⚠️  Para visualizar o diagrama, execute este código em um ambiente com matplotlib configurado.")

### 📋 Detalhamento dos Relacionamentos

#### 🔑 Chaves e Relacionamentos:

| Tabela | Chave Primária | Chaves Estrangeiras | Relacionamento |
|--------|----------------|-------------------|----------------|
| **fact_sales** | order_key | date_key, customer_key, product_key, seller_key | Fato Central |
| **dim_date** | date_key | - | 1:N com fact_sales |
| **dim_customer** | customer_id | - | 1:N com fact_sales |
| **dim_product** | product_id | - | 1:N com fact_sales |
| **dim_seller** | seller_id | - | 1:N com fact_sales |

#### 🎯 Lógica de Negócio dos Relacionamentos:

**1. fact_sales ➤ dim_date (1:N)**
- Um registro de venda acontece em uma data específica
- Uma data pode ter múltiplas vendas
- Permite análises temporais (vendas por dia/mês/ano)

**2. fact_sales ➤ dim_customer (1:N)**  
- Cada venda pertence a um cliente específico
- Um cliente pode fazer múltiplas compras
- Permite análises de comportamento do cliente

**3. fact_sales ➤ dim_product (1:N)**
- Cada linha de venda refere-se a um produto
- Um produto pode ser vendido múltiplas vezes
- Permite análises de performance de produtos

**4. fact_sales ➤ dim_seller (1:N)**
- Cada venda é processada por um vendedor
- Um vendedor pode processar múltiplas vendas
- Permite análises de performance de vendedores

#### 🛠️ Implementação Técnica:

```sql
-- Exemplo de consulta utilizando o modelo Star Schema
SELECT 
    d.year,
    d.month_name,
    c.customer_segment,
    p.category,
    s.seller_performance,
    SUM(f.total_amount) as revenue,
    COUNT(f.order_key) as orders,
    AVG(f.total_amount) as avg_order_value
FROM fact_sales f
JOIN dim_date d ON f.date_key = d.date_key
JOIN dim_customer c ON f.customer_key = c.customer_id  
JOIN dim_product p ON f.product_key = p.product_id
JOIN dim_seller s ON f.seller_key = s.seller_id
WHERE f.order_status = 'DELIVERED'
GROUP BY d.year, d.month_name, c.customer_segment, p.category, s.seller_performance
ORDER BY revenue DESC;
```

#### 📊 Vantagens do Modelo Star Schema:

✅ **Performance:** Consultas rápidas com poucos JOINs  
✅ **Simplicidade:** Estrutura fácil de entender  
✅ **Flexibilidade:** Permite análises multidimensionais  
✅ **Escalabilidade:** Otimizado para grandes volumes de dados  
✅ **BI-Ready:** Compatível com ferramentas de Business Intelligence  

#### 🔧 Considerações de Implementação:

- **Índices:** Criar índices nas chaves primárias e estrangeiras
- **Particionamento:** Particionar fact_sales por data para melhor performance
- **Compressão:** Utilizar compressão Delta Lake para otimizar storage
- **Manutenção:** Implementar rotinas de limpeza e otimização periódica

In [ ]:
# Geração de DER Simplificado em Texto (ASCII Art)
def generate_ascii_erd():
    """
    Gera uma representação ASCII do DER para visualização em ambientes sem matplotlib
    """
    
    ascii_erd = """
    
    🏗️ DIAGRAMA ENTIDADE-RELACIONAMENTO (DER) - AMAZON SALES MVP
    ═══════════════════════════════════════════════════════════════
    
    ┌─────────────────────────────────────────────────────────────────────────────┐
    │                           📅 DIM_DATE                                       │
    │  ┌─────────────────────────────────────────────────────────────────────┐   │
    │  │ 🔑 date_key (PK)    │ order_date     │ year        │ quarter      │   │
    │  │ month              │ month_name      │ day         │ day_of_week  │   │
    │  │ day_name           │ is_weekend      │             │              │   │
    │  └─────────────────────────────────────────────────────────────────────┘   │
    └─────────────────────────────────┬───────────────────────────────────────────┘
                                      │ 1:N
                                      ▼
    ┌──────────────────────────┐  ┌─────────────────────────────────────────────┐  ┌──────────────────────────┐
    │      👤 DIM_CUSTOMER      │  │               💰 FACT_SALES                │  │      📦 DIM_PRODUCT       │
    │ ┌─────────────────────────┤  │  ┌─────────────────────────────────────────┤  │ ┌─────────────────────────┤
    │ │🔑 customer_id (PK)     │◄─┤1:N│ order_key                              │N:1├─►│🔑 product_id (PK)      │
    │ │ customer_name          │  │  │🔗 date_key (FK)                        │  │ │ product_name           │
    │ │ city                   │  │  │🔗 customer_key (FK)                    │  │ │ category               │
    │ │ state                  │  │  │🔗 product_key (FK)                     │  │ │ brand                  │
    │ │ country                │  │  │🔗 seller_key (FK)                      │  │ │ avg_unit_price         │
    │ │ first_order_date       │  │  │ ─────────────────────                   │  │ │ total_sales            │
    │ │ last_order_date        │  │  │ quantity           │ unit_price         │  │ │ total_quantity_sold    │
    │ │ total_orders           │  │  │ gross_amount       │ discount_rate      │  │ │ price_category         │
    │ │ total_lifetime_value   │  │  │ discount_amount    │ tax_amount         │  │ └─────────────────────────┘
    │ │ customer_segment       │  │  │ shipping_cost      │ net_amount         │  └──────────────────────────┘
    │ └─────────────────────────┘  │  │ total_amount       │                    │
    └──────────────────────────────┘  │ ─────────────────────                   │
                                      │ payment_method     │ order_status       │
                                      │ is_suspicious_amt  │ is_failed_order    │
                                      │ fact_created_timestamp                  │
                                      └─────────────────────────────────────────┤
                                                       N:1 │                    │
                                                          ▼                     │
                                      ┌─────────────────────────────────────────┘
                                      │               🏪 DIM_SELLER              
                                      │  ┌─────────────────────────────────────┐ 
                                      │  │🔑 seller_id (PK)    │ unique_customers│ 
                                      │  │ unique_products     │ total_orders   │ 
                                      │  │ total_revenue       │ avg_order_value │ 
                                      │  │ seller_performance  │                │ 
                                      │  └─────────────────────────────────────┘ 
                                      └─────────────────────────────────────────
    
    ══════════════════════════════════════════════════════════════════════════════
    
    🔗 RELACIONAMENTOS:
    ├─ fact_sales ──► dim_date      (N:1) │ Muitos pedidos por data
    ├─ fact_sales ──► dim_customer  (N:1) │ Muitos pedidos por cliente  
    ├─ fact_sales ──► dim_product   (N:1) │ Muitos pedidos por produto
    └─ fact_sales ──► dim_seller    (N:1) │ Muitos pedidos por vendedor
    
    🎯 CARACTERÍSTICAS:
    • Modelo: Star Schema Clássico
    • Granularidade: Item de pedido (order line level)
    • Tabelas: 1 Fato + 4 Dimensões = 5 tabelas Gold
    • Métricas: 9 medidas aditivas na tabela fato
    • Otimizado para: Consultas analíticas OLAP
    
    ══════════════════════════════════════════════════════════════════════════════
    """
    
    return ascii_erd

# Exibir o DER em formato ASCII
print(generate_ascii_erd())

# Função para validar integridade do modelo
def validate_model_integrity():
    """
    Função para validar a integridade referencial do modelo
    Pode ser executada no ambiente Databricks
    """
    
    validation_queries = {
        "orphan_fact_records": """
            -- Verificar registros órfãos na tabela fato
            SELECT 'FACT_SALES' as table_name, COUNT(*) as orphan_count
            FROM fact_sales f
            LEFT JOIN dim_date d ON f.date_key = d.date_key
            LEFT JOIN dim_customer c ON f.customer_key = c.customer_id
            LEFT JOIN dim_product p ON f.product_key = p.product_id  
            LEFT JOIN dim_seller s ON f.seller_key = s.seller_id
            WHERE d.date_key IS NULL 
               OR c.customer_id IS NULL 
               OR p.product_id IS NULL 
               OR s.seller_id IS NULL
        """,
        
        "duplicate_keys": """
            -- Verificar chaves duplicadas nas dimensões
            SELECT 'DIM_DATE' as dimension, COUNT(*) - COUNT(DISTINCT date_key) as duplicates
            FROM dim_date
            UNION ALL
            SELECT 'DIM_CUSTOMER', COUNT(*) - COUNT(DISTINCT customer_id)
            FROM dim_customer  
            UNION ALL
            SELECT 'DIM_PRODUCT', COUNT(*) - COUNT(DISTINCT product_id)
            FROM dim_product
            UNION ALL  
            SELECT 'DIM_SELLER', COUNT(*) - COUNT(DISTINCT seller_id)
            FROM dim_seller
        """,
        
        "referential_integrity": """
            -- Verificar integridade referencial completa
            SELECT 
                'REFERENTIAL_INTEGRITY' as check_name,
                CASE 
                    WHEN orphan_count = 0 THEN '✅ PASSOU'
                    ELSE '❌ FALHOU'  
                END as status,
                orphan_count
            FROM (
                SELECT COUNT(*) as orphan_count
                FROM fact_sales f
                WHERE NOT EXISTS (SELECT 1 FROM dim_date d WHERE f.date_key = d.date_key)
                   OR NOT EXISTS (SELECT 1 FROM dim_customer c WHERE f.customer_key = c.customer_id)
                   OR NOT EXISTS (SELECT 1 FROM dim_product p WHERE f.product_key = p.product_id)
                   OR NOT EXISTS (SELECT 1 FROM dim_seller s WHERE f.seller_key = s.seller_id)
            )
        """
    }
    
    print("🔍 CONSULTAS DE VALIDAÇÃO DO MODELO:")
    print("=" * 50)
    
    for check_name, query in validation_queries.items():
        print(f"\n📋 {check_name.upper().replace('_', ' ')}:")
        print("─" * 40)
        print(query)
    
    print("\n✅ Execute estas consultas no Databricks para validar a integridade do modelo!")

validate_model_integrity()